Spectral Information Extraction<br>
This script is used to extract information from image containing spectral bands for building the relationship between LAI and vegetation indices.<br>
Required inputs:<br>
> (1) Grids in one shapefile;<br>
> (2) The image with spectral bands.

The output is a table in "*.csv" format, and it contains the Grid 'FID' (starts from 1 instead of 0, not ID), the average value for each 6 bands, and the fractional cover.<br>
Contact information:<br>
Rui Gao<br>
rui.gao@aggiemail.usu.edu

In [1]:
import arcpy
import arcview
from arcpy import env
import os
import gdal
import shutil
import numpy as np
import pandas as pd
import shapefile
import matplotlib.pyplot as plt
import datetime

C:\Users\grui9\anaconda3\envs\TSEB\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Product and extension licensing is no longer handled with this method.
  


In [2]:
def FolderCreate(NewFolderPath):
    if not os.path.exists(NewFolderPath):
        os.makedirs(NewFolderPath)

# The number of ground LAI measurements

In [3]:
LAI_Grids = r'D:\Project_Wellsville\7_LAI_VIs_Relation_1_meter_Grid\1_Coordinates_Ground_Measurements\2_ArcGIS_Grids\5R_0904.shp'
shp_info = shapefile.Reader(LAI_Grids)
records = shp_info.records()
num_records = len(records)
print("Totally",num_records,"ground LAI measurements.")

Totally 345 ground LAI measurements.


# Parameters, path setting

In [4]:
# information about the shapefile
dir_tmp = r'D:\Project_Wellsville\7_LAI_VIs_Relation_1_meter_Grid\1_Coordinates_Ground_Measurements\3_Extraction\5R_0904'
FolderCreate(dir_tmp)
arcpy.env.overwriteOutput = True
Shp_Grid_FileName = "FocusOnThisGrid.shp"
noDataValue = np.nan

# information about image. This image is resampled one (10 cm for my case)
dir_image = r'D:\Project_Wellsville\4_LAI_VI_Relationship\1_ArcGIS_Pro_Well\Clip_Raster.tif'

In [5]:
band_B = 0; band_G = 1; band_R = 2; band_RE = 3; band_NIR = 4; band_Tr = 5
ID = []
B = []
G = []
R = []
RE = []
NIR = []
Tr = []
fc = []
for ishp in range(num_records):
    env.workspace = LAI_Grids
    Shapefile = LAI_Grids.replace('.shp', '_lyr')
    arcpy.MakeFeatureLayer_management(LAI_Grids, Shapefile)
    arcpy.SelectLayerByAttribute_management(Shapefile, 'NEW_SELECTION', '"FID" =' + str(ishp))
    arcpy.CopyFeatures_management(Shapefile,
                                  dir_tmp + "\\" + Shp_Grid_FileName)
    
    # read the matrix for each location
    out_raster = arcpy.gp.ExtractByMask_sa(dir_image, dir_tmp + "\\" + Shp_Grid_FileName, 
                                           dir_tmp+"\\"+"GridID_"+str(ishp)+".tif")
    array_grid = arcpy.RasterToNumPyArray(dir_tmp+"\\"+"GridID_"+str(ishp)+".tif", 
                                          nodata_to_value=noDataValue)
    
    # calculate the fractional cover
    array_ndvi = (array_grid[band_NIR,:,:]-array_grid[band_R,:,:])/(array_grid[band_NIR,:,:]+array_grid[band_R,:,:])
    array_ndvi[array_ndvi>=0.4] = 1; array_ndvi[array_ndvi<1] = 0

    # get the average for each band
    ID.append(ishp+1)
    B.append(np.nanmean(array_grid[band_B,:,:]))
    G.append(np.nanmean(array_grid[band_G,:,:]))
    R.append(np.nanmean(array_grid[band_R,:,:]))
    RE.append(np.nanmean(array_grid[band_RE,:,:]))
    NIR.append(np.nanmean(array_grid[band_NIR,:,:]))
    Tr.append(np.nanmean(array_grid[band_Tr,:,:]))
    fc.append(np.nansum(array_ndvi)/(array_ndvi.shape[0]*array_ndvi.shape[1]))
    print("Processed:",str(ishp+1),'/',num_records)


Processed: 1 / 345
Processed: 2 / 345
Processed: 3 / 345
Processed: 4 / 345
Processed: 5 / 345
Processed: 6 / 345
Processed: 7 / 345
Processed: 8 / 345
Processed: 9 / 345
Processed: 10 / 345
Processed: 11 / 345
Processed: 12 / 345
Processed: 13 / 345
Processed: 14 / 345
Processed: 15 / 345
Processed: 16 / 345
Processed: 17 / 345
Processed: 18 / 345
Processed: 19 / 345
Processed: 20 / 345
Processed: 21 / 345
Processed: 22 / 345
Processed: 23 / 345
Processed: 24 / 345
Processed: 25 / 345
Processed: 26 / 345
Processed: 27 / 345
Processed: 28 / 345
Processed: 29 / 345
Processed: 30 / 345
Processed: 31 / 345
Processed: 32 / 345
Processed: 33 / 345
Processed: 34 / 345
Processed: 35 / 345
Processed: 36 / 345
Processed: 37 / 345
Processed: 38 / 345
Processed: 39 / 345
Processed: 40 / 345
Processed: 41 / 345
Processed: 42 / 345
Processed: 43 / 345
Processed: 44 / 345
Processed: 45 / 345
Processed: 46 / 345
Processed: 47 / 345
Processed: 48 / 345
Processed: 49 / 345
Processed: 50 / 345
Processed

In [6]:
df_out = pd.DataFrame()
df_out['ID'] = ID
df_out['B'] = B
df_out['G'] = G
df_out['R'] = R
df_out['RE'] = RE
df_out['NIR'] = NIR
df_out['Tr'] = Tr
df_out['Fc'] = fc
df_out
df_out.to_csv(dir_tmp+'\\5R_0904.csv',index=False)

In [7]:
df_out

,ID,B,G,R,RE,NIR,Tr,Fc
0,1,0.020698,0.049003,0.025740,0.105043,0.544060,23.544512,0.826446
1,2,0.025576,0.055462,0.030621,0.115797,0.564817,23.908310,0.826446
2,3,0.030279,0.061788,0.038969,0.125242,0.503482,24.518408,0.826446
3,4,0.021356,0.053820,0.023748,0.118030,0.567670,24.829609,0.826446
4,5,0.022636,0.051324,0.030236,0.107112,0.420886,23.679209,0.826446
...,...,...,...,...,...,...,...,...
340,341,0.034647,0.079859,0.051428,0.198206,0.485735,27.342812,0.826446
341,342,0.026583,0.065757,0.029603,0.170983,0.480150,25.488510,0.826446
342,343,0.027490,0.067929,0.027781,0.177515,0.505579,24.407310,0.826446
343,344,0.023598,0.063491,0.020511,0.175815,0.556593,23.949009,0.826446
